In [1]:
# *************************************************************************
#
# Royal Dutch Shell
# __________________
#
# Copyright (c) 2022, Royal Dutch shell. All Rights Reserved.
#
# NOTICE: All information contained herein is, and remains
# the property of Royal Dutch Shell and its suppliers,
# if any. The intellectual and technical concepts contained
# herein are proprietary to Royal Dutch Shell
# and its suppliers and may be covered by U.S. and Foreign Patents,
# patents in process, and are protected by trade secret or copyright law.
# Dissemination of this information or reproduction of this material
# is strictly forbidden unless prior written permission is obtained
# from Royal Dutch Shell.
# *************************************************************************

In [2]:
'''Import Libraries'''
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook
from tqdm import tqdm
import time
import warnings
from src.global_variables import *
from src.data.dataloader import load_solar_profile
from src.utilities import *
warnings.filterwarnings("ignore")

/glb/hou/gf.siti/data/rl_sfo/anaconda3/envs/rl-solar/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pyomo Model

In [3]:
from pyomo.opt import TerminationCondition
from pyomo.opt import SolverStatus
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.environ import ConcreteModel, Set, Var, Constraint, Objective, maximize, minimize, Binary, Boolean

In [11]:
'''Creater Solver Instance through SolverFactory using ipopt solver'''

opt       = SolverFactory('glpk')    #cplex  #gurobi  #cbc #scip #glpk
stream_solver = False              # For readable Output
keepfiles     = False              # For output record
episode_lenth = 744

In [5]:
data_path = '/glb/hou/gf.siti/data/rl_sfo/luiz_code/'\
            'solar_fuel_ReinforcementLearning/data/solar_profile_hourlyV2.csv'
power_supply_df = load_solar_profile(data_path=data_path, 
                                     year="TMY P90", 
                                     initial_month='Jan', 
                                     number_of_months=1)
hourly_demand = calculate_hourly_demand(whole_year_supply=power_supply_df,
                                       total_demand=64400,
                                       buffer_demand=10000)

Whole Year Supply DF description
             hour        volume
count  744.000000    744.000000
mean   372.500000   2342.088198
std    214.918589   3237.723397
min      1.000000      0.000000
25%    186.750000      0.000000
50%    372.500000      0.000000
75%    558.250000   5513.913680
max    744.000000  10303.159520
The sum of the whole energy volume is 37752422.82975999
Total duration 744
Number of hours without sun: 15
Total Demand: 64400
Buffer Demand: 5000
Hourly Buffer Demand: 858.6666666666666
Remain Demand: 51520.0
Total demand per day integrating the profile: 2000320.0


In [12]:
model = ConcreteModel()

model.solar_to_battery = Var([i for i in range(episode_lenth)], initialize= 0, bounds=(0, MAX_SOLAR_IN)) 
model.solar_to_electrolyzer = Var([i for i in range(episode_lenth)], initialize= 0, bounds=(0, MAX_SOLAR_IN))
model.battery_to_electrolyzer = Var([i for i in range(episode_lenth)], initialize= 0, bounds=(0, MAX_BATTERY_STORAGE_CAPACITY))
model.solar_to_curtail = Var([i for i in range(episode_lenth)], initialize= 0, bounds=(0, MAX_SOLAR_IN))

model.solar_in = Var([i for i in range(episode_lenth)], initialize= 0, bounds=(0, MAX_SOLAR_IN))
model.solar_in[0].fixed = True

model.battery_level = Var([i for i in range(episode_lenth)], initialize=MAX_BATTERY_STORAGE_CAPACITY, bounds=(0, MAX_BATTERY_STORAGE_CAPACITY))
model.battery_level[0].fixed = True

model.electrolyzer = Var([i for i in range(episode_lenth)], initialize=0, bounds=(0, 99999999999999))
model.electrolyzer[0].fixed = True

def my_constr1(m, i):
    return (m.battery_level[i] - (m.battery_level[i-1] + model.solar_to_battery[i-1] - \
                                              - model.battery_to_electrolyzer[i-1]) == 0)
model.C1 = Constraint([i for i in range(1,episode_lenth)], rule = my_constr1)

def my_constr2(m, i):
    return (m.electrolyzer[i] - (m.solar_to_electrolyzer[i-1] + model.battery_to_electrolyzer[i-1]) == 0)
model.C2 = Constraint([i for i in range(1,episode_lenth)], rule = my_constr2)

def my_constr3(m, i):
    return (m.solar_in[i] - (m.solar_to_electrolyzer[i-1] + model.solar_to_battery[i-1] - \
                                          model.solar_to_curtail[i-1]) == 0)
model.C3 = Constraint([i for i in range(1,episode_lenth)], rule = my_constr3)


def obj_fun_rule(m):
    
    prod_unit_cost        =  0
    charging_cost      =  0
    discharging_cost       =  0
    terminal_storage_cost =  0
    battery_storage_cost     = 0
    curtail_cost = 5

    # selling price of product. Assumed fixed
    prod_unit_price = 5
    
    reward = 0
    for i in range(744):
        demand_t = hourly_demand[i]
        
        if m.electrolyzer[i].value == demand_t:
            total_revenue = demand_t*prod_unit_price
            total_penalty_cost = 0
        else:
            total_revenue = m.electrolyzer[i]*prod_unit_price  # because will sell this much
            total_penalty_cost = abs(demand_t - m.electrolyzer[i])*prod_unit_price
        
        total_prod_cost = m.solar_to_electrolyzer[i]*prod_unit_cost
        total_charging_cost = m.solar_to_battery[i]*charging_cost
        total_dischargin_cost = m.battery_to_electrolyzer[i]*discharging_cost
        total_battery_cost = m.battery_level[i]*battery_storage_cost
        total_curtail_cost = 0 #m.solar_to_curtail[i]*curtail_cost
        
        reward += total_revenue - total_prod_cost - total_charging_cost - total_dischargin_cost - \
                 total_battery_cost - total_curtail_cost - total_penalty_cost
        
    return(reward)
        
model.obj_fun = Objective(rule = obj_fun_rule, sense=maximize)

results = opt.solve(model, tee=stream_solver)
results

RuntimeError: Cannot write legal LP file.  Objective 'obj_fun' has nonlinear terms that are not quadratic.

## Profit and Cumulative Profit

In [ ]:
demand_profile = pd.read_csv(r'C:\Users\Nitin.N.Singh\Desktop\DRL SCO\Data\demand_2021_tz.csv')['volume']

profit = []

prod_unit_cost        =  40
stockpiling_cost      =  5
reclaiming_cost       =  5
shipping_cost         =  20
terminal_storage_cost =  0.16
silo_storage_cost     = 0.01

# selling price of product. Assumed fixed
prod_unit_price = 150

# production and demand
daily_supply = 1600
    
for i in range(365):
    demand_t = demand_profile[i]
        
    if model.terminal_storage[i].value >= demand_t:
        total_revenue = demand_t*prod_unit_price
        total_penalty_cost = 0
    else:
        total_revenue = model.terminal_storage[i].value*prod_unit_price  # because will sell this much
        total_penalty_cost = (demand_t - model.terminal_storage[i].value)*prod_unit_price

    total_prod_cost        = daily_supply*prod_unit_cost
    total_stockpiling_cost = model.stockpiling[i].value*stockpiling_cost
    total_reclaiming_cost  = model.reclaiming[i].value*reclaiming_cost
    total_shipping_cost    = model.shipping[i].value*shipping_cost
    total_silo_storage_cost     = silo_storage_cost*model.silo_storage[i].value
    total_terminal_storage_cost = terminal_storage_cost*model.terminal_storage[i].value

    reward = total_revenue - total_prod_cost - total_stockpiling_cost - total_reclaiming_cost - \
             total_shipping_cost - total_silo_storage_cost - total_terminal_storage_cost - total_penalty_cost
    profit.append(reward)
    
plt.figure(figsize=(10,3)) 
plt.plot(profit) 
plt.ylabel('Profit')

In [ ]:
print("\x1b[34m >>>>>>>>> Cumulative Profit:  \x1b[0m", np.cumsum(profit)[-1])

plt.figure(figsize=(10,3)) 
plt.plot(np.cumsum(profit)) 
plt.ylabel('Cumulative Profit')

### Storages and Shippings

In [ ]:
plt.figure(figsize=(10,3))
to_plot = []
for i in range(365):
    to_plot.append(model.reclaiming[i].value)
plt.plot(to_plot) 
plt.ylabel('Reclaiming')

plt.figure(figsize=(10,3))
to_plot = []
for i in range(365):
    to_plot.append(model.stockpiling[i].value)
plt.plot(to_plot) 
plt.ylabel('Stockpiling')

plt.figure(figsize=(10,3))
to_plot = []
for i in range(365):
    to_plot.append(model.shipping[i].value)
plt.plot(to_plot, label = 'Shipping')
plt.plot(demand_profile, label = 'Demand')
plt.legend()

In [ ]:
plt.figure(figsize=(10,3))
to_plot = []
for i in range(365):
    to_plot.append(model.silo_storage[i].value)
plt.plot(to_plot, label = 'Silo Storage') 
plt.ylabel('Silo Storage')

plt.figure(figsize=(10,3))
to_plot = []
for i in range(365):
    to_plot.append(model.stockpile_storage[i].value)
plt.plot(to_plot, label = 'Stockpile Storage') 
plt.ylabel('Stockpile Storage')

plt.figure(figsize=(10,3))
to_plot = []
for i in range(365):
    to_plot.append(model.terminal_storage[i].value)
plt.plot(to_plot, label = 'Terminal Storage') 
plt.plot(demand_profile, label = 'Demand')

plt.legend()